In [1]:
# Prerequisite: Installs the latest version of Ludwig in the Colab environment
!python -m pip install git+https://github.com/ludwig-ai/ludwig.git --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 136 kB 8.7 MB/s 
     |████████████████████████████████| 47 kB 4.3 MB/s 
     |████████████████████████████████| 398 kB 42.3 MB/s 
     |████████████████████████████████| 1.1 MB 42.5 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |████████████████████████████████| 144 kB 42.8 MB/s 
     |████████████████████████████████| 271 kB 37.1 MB/s 


In [63]:
!ludwig datasets download adult_census_income

NumExpr defaulting to 2 threads.
███████████████████████
█ █ █ █  ▜█ █ █ █ █   █
█ █ █ █ █ █ █ █ █ █ ███
█ █   █ █ █ █ █ █ █ ▌ █
█ █████ █ █ █ █ █ █ █ █
█     █  ▟█     █ █   █
███████████████████████
ludwig v0.5rc2 - Datasets download



In [64]:
import pandas as pd
import numpy as np

raw_df = pd.read_csv('./adult_census_income.csv')
raw_df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income,split
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,0


In [73]:
config_yaml = """
preprocessing:
  number:
    normalization: zscore
    missing_value_strategy: fill_with_mean

input_features:
  - name: age
    type: number
  - name: workclass
    type: category
  - name: fnlwgt
    type: number
  - name: education
    type: category
  - name: education-num
    type: number
  - name: marital-status
    type: category
  - name: occupation
    type: category
  - name: relationship
    type: category
  - name: race
    type: category
  - name: sex
    type: category
  - name: capital-gain
    type: number
  - name: capital-loss
    type: number
  - name: hours-per-week
    type: number
  - name: native-country
    type: category

combiner:
  type: concat
  num_fc_layers: 3
  output_size: 128
  dropout: 0.2

output_features:
  - name: income
    type: binary
    preprocessing:
      fallback_true_label: " >50K"
    num_fc_layers: 4
    output_size: 32

trainer:
  epochs: 5
  optimizer: 
    type: sgd
"""

# Writes config to a file called "config.yaml"
with open("config.yaml", "w") as f:
  f.write(config_yaml)

In [74]:
!rm -fr results test_results visualizations

In [75]:
# Trains the model. This cell might take a few minutes.
!ludwig experiment --dataset adult_census_income.csv \
              --config config.yaml \
              --logging_level warning

NumExpr defaulting to 2 threads.
import ray failed with exception: No module named 'ray'
Binary feature income has at least 1 unconventional boolean value: Cannot automatically map value  <=50K to a boolean and no `fallback_true_label` specified.. We will now interpret  >50K as 1 and the other values as 0. If this is incorrect, please use the category feature type or manually specify the true value with `preprocessing.fallback_true_label`.
Note: steps_per_checkpoint (was 255) is now set to the number of steps per epoch: 255.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [68]:
# Extract subset of test data for evaluation due to limitations in amount of data displayable in colab notebook.
#np.random.seed(13)
#raw_df.loc[raw_df.split == 2].sample(n=200).to_csv('evaluation_dataset.csv', index=False)

In [69]:
# Generates predictions and performance statistics for the test set.
# !ludwig evaluate --model_path results/experiment_run/model \
#                  --dataset evaluation_dataset.csv \
#                  --split full \
#                  --output_directory test_results

# Visualize Metrics

In [76]:
!ludwig visualize --visualization roc_curves \
                  --ground_truth adult_census_income.csv \
                  --split_file adult_census_income.split.csv \
                  --ground_truth_metadata results/experiment_run/model/training_set_metadata.json \
                  --probabilities results/experiment_run/predictions.parquet \
                  --output_feature_name income \
                  --output_directory visualizations \
                  --model_names Model1 \
                  --file_format png

                  # --positive_label 2 \


NumExpr defaulting to 2 threads.
import ray failed with exception: No module named 'ray'
Traceback (most recent call last):
  File "/usr/local/bin/ludwig", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/ludwig/cli.py", line 158, in main
    CLI()
  File "/usr/local/lib/python3.7/dist-packages/ludwig/cli.py", line 63, in __init__
    getattr(self, args.command)()
  File "/usr/local/lib/python3.7/dist-packages/ludwig/cli.py", line 98, in visualize
    visualize.cli(sys.argv[2:])
  File "/usr/local/lib/python3.7/dist-packages/ludwig/visualize.py", line 3823, in cli
    vis_func(**vars(args))
  File "/usr/local/lib/python3.7/dist-packages/ludwig/visualize.py", line 1022, in roc_curves_cli
    **kwargs,
  File "/usr/local/lib/python3.7/dist-packages/ludwig/visualize.py", line 3058, in roc_curves
    ground_truth = _vectorize_ground_truth(ground_truth, feature_metadata["str2idx"], ground_truth_apply_idx)
KeyError: 'str2idx'
